# Portfolio Analytics 
### Logan Chalifour

---

1. Given a folder of Excel files and tasked with creating a dataset & visualization to help investigate trends.  
2. The Excel data is purposefully anonymized, and the row orders are random across files.  
3. Each row of data in the Excel file represents a Fund.  
4. None of the Excel funds are Fund Apple, Banana, or Cherry.  
5. Outside of the Excel data, we have these facts:

| Fund    | Vintage | End Date   | IRR    | TVPI  |
|---------|---------|------------|--------|-------|
| Apple   | 2012    | 12/31/2022 | 14.7%  | 1.97  |
| Banana  | 2016    | 12/31/2022 | 19.5%  | 2.12  |
| Cherry  | 2019    | 12/31/2022 | 14.8%  | 1.22  |

---

## Setup

I begin by downloading the ZIP file and saving it locally. To better understand the data structure, I open several files directly in Excel.

There are 36 Excel files in total, each named following the pattern `export_{file number}.xlsx`. Each file contains three sheets that are generally consistent in structure, though I plan to add handling for any deviations that may arise:

- **Data**: Contains the core fund-level metrics, including `x` and `y` columns, which I will use for visualization.  
- **Filter**: Lists filtering criteria applied during export, structured as key-value pairs with columns `Criteria` and `Value`. Common fields include `Vintage` and `Asset Class`. For example, from `export_1`:

  | Criteria    | Value   |
  |-------------|---------|
  | Vintage     | 2012    |
  | Asset Class | (All)   |

- **Parameters**: Holds metadata about the export as key-value pairs in columns `Parameter` and `Value`. Key parameters include axis labels and date information. For example, from `export_1`:

  | Parameter      | Value       |
  |----------------|-------------|
  | End Date       | 12/31/2020  |
  | x-axis Measure | IRR         |
  | y-axis Measure | TVPI        |

After familiarizing myself with this structure, I set up this Python virtual environment to dive in further.

---

## Install Packages

We begin by importing the necessary packages below.

In [1]:
# Install required packages from the requirements file
!pip install -r requirements.txt
!pip install openpyxl==3.1.5

# Import necessary libraries for data processing and visualization
import os
import openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

Processing /deepnote-toolkit/dist/deepnote_toolkit-0.1.0-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/deepnote-toolkit/dist/deepnote_toolkit-0.1.0-py3-none-any.whl'


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


---

## ETL and EDA

Next, we read the Excel files and store them in a dictionary. To flatten the data, I append values from the `Filter` and `Parameters` sheets as additional columns alongside the `x` and `y` values from the `Data` sheet.

In [2]:
folder_path = "exports"  # Local folder containing Excel files
expected_sheets = ["Data", "Filter", "Parameters"]  # Expected sheet names for each file
export_dfs = {}  # Dictionary to store processed DataFrames keyed by filename

# Verify the folder exists
if os.path.exists(folder_path) and os.path.isdir(folder_path):

    exports = os.listdir(folder_path)  # List all files in the folder

    # Iterate through each file in the folder
    for file in exports:

        # Process only Excel files
        if file.endswith(".xlsx"):

            file_name = os.path.splitext(file)[0]  # Extract filename without extension
            file_path = os.path.join(folder_path, file)  # Full path to file

            try:
                # Read all sheets from the Excel file
                excel_data = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")
                sheet_names = list(excel_data.keys())

                # Confirm sheet names and order match expected structure
                if sheet_names == expected_sheets:

                    # Load each sheet into a df
                    data_df = excel_data["Data"]
                    filter_df = excel_data["Filter"]
                    param_df = excel_data["Parameters"]

                    # Convert Filter and Parameters sheets to metadata dictionaries
                    filter_dict = pd.Series(filter_df.Value.values, index=filter_df.Criteria).to_dict()
                    param_dict = pd.Series(param_df.Value.values, index=param_df.Parameter).to_dict()

                    # Merge metadata dictionaries
                    meta_dict = {**param_dict, **filter_dict}

                    # Append metadata as new columns to the data df, broadcasted to all rows
                    for key, value in meta_dict.items():
                        data_df[key] = value

                    # Store the processed df in the dictionary
                    export_dfs[file_name] = data_df
                    print(f"Processed: {file_name}")

                # If sheet names do not match expected structure
                else:
                    
                    print(f"Warning: {file} does not have the required sheets with the correct names or order. Skipping.")

            # Catch any other exceptions
            except Exception as e:
                print(f"Failed to process {file_name}: {e}")

# If the folder does not exist
else:
    print(f'"{folder_path}" folder does not exist.')


Processed: export_1
Processed: export_10
Processed: export_11
Processed: export_12
Processed: export_13
Processed: export_14
Processed: export_15
Processed: export_16
Processed: export_17
Processed: export_18
Processed: export_19
Processed: export_2
Processed: export_20
Processed: export_21
Processed: export_22
Processed: export_23
Processed: export_24
Processed: export_25
Processed: export_26
Processed: export_27
Processed: export_28
Processed: export_29
Processed: export_3
Processed: export_30
Processed: export_31
Processed: export_32
Processed: export_33
Processed: export_34
Processed: export_35
Processed: export_36
Processed: export_4
Processed: export_5
Processed: export_6
Processed: export_7
Processed: export_8
Processed: export_9


Below, I confirm that all 36 files were successfully processed by checking the length of the `export_dfs` dictionary.

In [3]:
len(export_dfs)

36

Using `export_1` as an example, I examine how one of these files is structured within the `export_dfs` dictionary.

In [4]:
export_dfs['export_1']

,x,y,End Date,x-axis Measure,y-axis Measure,Vintage,Asset Class
0,8.76,1.47,12/31/2020,IRR,TVPI,2012,(All)
1,4.29,1.16,12/31/2020,IRR,TVPI,2012,(All)
2,5.62,1.17,12/31/2020,IRR,TVPI,2012,(All)
3,16.44,1.94,12/31/2020,IRR,TVPI,2012,(All)
4,8.31,1.29,12/31/2020,IRR,TVPI,2012,(All)
...,...,...,...,...,...,...,...
346,11.23,1.93,12/31/2020,IRR,TVPI,2012,(All)
347,9.10,1.30,12/31/2020,IRR,TVPI,2012,(All)
348,17.69,2.12,12/31/2020,IRR,TVPI,2012,(All)
349,12.88,2.15,12/31/2020,IRR,TVPI,2012,(All)


I then combine all the 36 DataFrames into a singular `combined_df`, which contains nearly 20,000 rows, each representing a fund at a given point in time.

In [5]:
combined_df = pd.concat(export_dfs, ignore_index=True)
combined_df

,x,y,End Date,x-axis Measure,y-axis Measure,Vintage,Asset Class
0,8.76,1.47,12/31/2020,IRR,TVPI,2012,(All)
1,4.29,1.16,12/31/2020,IRR,TVPI,2012,(All)
2,5.62,1.17,12/31/2020,IRR,TVPI,2012,(All)
3,16.44,1.94,12/31/2020,IRR,TVPI,2012,(All)
4,8.31,1.29,12/31/2020,IRR,TVPI,2012,(All)
...,...,...,...,...,...,...,...
19210,3.10,1.22,6/30/2022,IRR,TVPI,2012,(All)
19211,29.68,2.64,6/30/2022,IRR,TVPI,2012,(All)
19212,10.01,1.49,6/30/2022,IRR,TVPI,2012,(All)
19213,9.82,1.95,6/30/2022,IRR,TVPI,2012,(All)


The ETL process described above is designed to handle any number of files in the export folder, as long as each file follows the expected structure with `Data`, `Filter`, and `Parameters` sheets. It is also flexible enough to accommodate changes in the `Filter` and `Parameters` sheets - for example, new rows for additional filters or parameters can be added without disrupting the job. This ensures that future files added to the export folder can be processed seamlessly.

Since each row in the dataset represents a fund, I next add the fruit funds `Apple`, `Banana`, and `Cherry` to the DataFrame for later comparison.


In [6]:
# Create a DataFrame for the fruit funds with their known metrics
benchmark_funds_df = pd.DataFrame([
    {
        "Fund Name": "Apple",
        "Vintage": 2012,
        "End Date": pd.to_datetime("12/31/2022"),
        "x": 14.7,    # IRR value
        "y": 1.97,    # TVPI value
        "x-axis Measure": "IRR",
        "y-axis Measure": "TVPI",
        "Highlight": True,
    },
    {
        "Fund Name": "Banana",
        "Vintage": 2016,
        "End Date": pd.to_datetime("12/31/2022"),
        "x": 19.5,
        "y": 2.12,
        "x-axis Measure": "IRR",
        "y-axis Measure": "TVPI",
        "Highlight": True,
    },
    {
        "Fund Name": "Cherry",
        "Vintage": 2019,
        "End Date": pd.to_datetime("12/31/2022"),
        "x": 14.8,
        "y": 1.22,
        "x-axis Measure": "IRR",
        "y-axis Measure": "TVPI",
        "Highlight": True,
    },
])

# Set defaults for non-benchmark funds
combined_df["Fund Name"] = "N/A"
combined_df["Highlight"] = False

# Append the benchmark fruit funds to the consolidated df
combined_df = pd.concat([combined_df, benchmark_funds_df], ignore_index=True)

I also update some data types and add new columns to the DataFrame:  
- `End Year`, extracted from the `End Date`  
- `Fund Age`, calculated as `End Year` minus `Vintage`

In [7]:
# Convert 'End Date' from string to datetime
combined_df['End Date'] = pd.to_datetime(combined_df['End Date'])

# Extract year from 'End Date' and store as a new column
combined_df['End Year'] = combined_df['End Date'].dt.year

# Calculate fund age as the difference between End Year and Vintage
combined_df['Fund Age'] = combined_df['End Year'] - combined_df['Vintage'].astype(int)
combined_df['Vintage'] = combined_df['Vintage'].astype(str)

With the consolidated DataFrame and new columns created, we now move on to investigating patterns in the data across both numerical and categorical variables.

In [8]:
# Create a temporary df for peer funds (excluding fruit funds)
peer_funds_df = combined_df[combined_df["Fund Name"] == "N/A"]

# Define numerical and categorical variable names
num_vars = ["x", "y", "Fund Age"]
cat_vars = [col for col in peer_funds_df.columns if col not in num_vars]

# Display summary statistics for numerical variables
print("=== Numerical Variables Summary ===")
print(peer_funds_df[num_vars].describe())
print("-" * 50)

# Display value counts for each categorical variable
print("\n=== Categorical Variables Breakdown ===")
for column in cat_vars:
    print(f"Column: {column}")
    print(peer_funds_df[column].value_counts(dropna=False))
    print("-" * 50)

=== Numerical Variables Summary ===
                  x             y      Fund Age
count  19215.000000  19215.000000  19215.000000
mean      16.555955      1.579608      4.519386
std       38.968016      1.596623      2.830051
min     -100.000000      0.000000      1.000000
25%        4.930000      1.080000      2.000000
50%       12.760000      1.310000      4.000000
75%       24.155000      1.700000      6.000000
max      985.280000     59.010000     10.000000
--------------------------------------------------

=== Categorical Variables Breakdown ===
Column: End Date
End Date
2021-09-30    1604
2022-09-30    1604
2021-12-31    1604
2022-12-31    1604
2021-03-31    1604
2022-03-31    1604
2021-06-30    1604
2022-06-30    1604
2020-12-31    1603
2020-09-30    1599
2020-06-30    1594
2020-03-31    1587
Name: count, dtype: int64
--------------------------------------------------
Column: x-axis Measure
x-axis Measure
IRR    19215
Name: count, dtype: int64
--------------------------------

Based on the summaries above, `IRR` and `TVPI` are the only distinct values for `x` and `y`, so I rename the columns accordingly and drop the redundant ones.

In [9]:
# Rename generic metric columns to their actual values
combined_df = combined_df.rename(columns={"x": "IRR", "y": "TVPI"})

# Drop redundant axis label columns now that the metrics have been renamed
combined_df = combined_df.drop(columns=["x-axis Measure", "y-axis Measure"])

I also identify three distinct `Vintage` years (2012, 2016, and 2019), which correspond to the benchmark funds labeled `Apple`, `Banana`, and `Cherry`. These may serve as peer groups for comparison, which we’ll explore further in the visualizations below.

For now, I create a dedicated `peer_funds_df` for the anonymized funds (i.e., those not labeled as `Apple`, `Banana`, or `Cherry`). I then create a `fruit_funds_df` to store the benchmark funds separately. The full dataset remains consolidated in the `combined_df` DataFrame for comparison.

In [10]:
# Create DataFrames for the peers and fruit funds
peer_funds_df = combined_df[combined_df["Fund Name"] == "N/A"]
fruit_funds_df = combined_df[combined_df["Fund Name"] != "N/A"]

In [11]:
combined_df

,IRR,TVPI,End Date,Vintage,Asset Class,Fund Name,Highlight,End Year,Fund Age
0,8.76,1.47,2020-12-31,2012,(All),N/A,False,2020,8
1,4.29,1.16,2020-12-31,2012,(All),N/A,False,2020,8
2,5.62,1.17,2020-12-31,2012,(All),N/A,False,2020,8
3,16.44,1.94,2020-12-31,2012,(All),N/A,False,2020,8
4,8.31,1.29,2020-12-31,2012,(All),N/A,False,2020,8
...,...,...,...,...,...,...,...,...,...
19213,9.82,1.95,2022-06-30,2012,(All),N/A,False,2022,10
19214,61.50,19.70,2022-06-30,2012,(All),N/A,False,2022,10
19215,14.70,1.97,2022-12-31,2012,NaN,Apple,True,2022,10
19216,19.50,2.12,2022-12-31,2016,NaN,Banana,True,2022,6


---

## Over Time Visualization

To better understand how private equity performance evolves over the fund lifecycle, I analyze the mean and median `IRR` and `TVPI` by `Fund Age`. Since `IRR` and `TVPI` are key metrics in the dataset, I refreshed my understanding of their definitions:

- **IRR (Internal Rate of Return)** – The _annualized_ rate of return that accounts for the timing and size of cash flows. It reflects how efficiently and quickly capital is returned to investors over the life of the fund.

- **TVPI (Total Value to Paid-In)** – A multiple that measures the total value generated by the fund relative to invested capital, combining both realized returns (distributions) and unrealized value (remaining NAV). It indicates the _overall_ value created for investors.

In the early years, PE fund returns can be low or even negative due to fees, uninvested capital, and time needed to create value within portfolio companies. As funds mature, especially between years 5 and 10, returns generally improve as investments mature and exits occur.

For this analysis, I included both average and median values to get a better sense of typical performance. The median helps mitigate the effect of extreme outliers, which were apparent in the upper echelon of funds.

Let's take a look to see how these hypotheses pan out below.

In [12]:
# Select colors for charts
color_a = "darkblue"
color_b = "lightsteelblue"

# Group by Fund Age and calculate mean and median for IRR and TVPI
age_summary = combined_df.groupby("Fund Age").agg(
    IRR_mean=("IRR", "mean"),
    IRR_median=("IRR", "median"),
    TVPI_mean=("TVPI", "mean"),
    TVPI_median=("TVPI", "median"),
).reset_index()

fig = go.Figure()

# Average IRR line (left y-axis, solid)
fig.add_trace(
    go.Scatter(
        x=age_summary["Fund Age"],
        y=age_summary["IRR_mean"],
        mode="lines+markers",
        name="Average IRR (%)",
        line=dict(color=color_a, width=2),
        yaxis="y1",
    )
)

# Median IRR trace line (left y-axis, dashed)
fig.add_trace(
    go.Scatter(
        x=age_summary["Fund Age"],
        y=age_summary["IRR_median"],
        mode="lines+markers",
        name="Median IRR (%)",
        line=dict(color=color_a, width=2, dash="dash"),
        yaxis="y1",
    )
)

# Average TVPI line (right y-axis, solid)
fig.add_trace(
    go.Scatter(
        x=age_summary["Fund Age"],
        y=age_summary["TVPI_mean"],
        mode="lines+markers",
        name="Average TVPI",
        line=dict(color=color_b, width=2),
        yaxis="y2",
    )
)

# Median TVPI line (right y-axis, dashed)
fig.add_trace(
    go.Scatter(
        x=age_summary["Fund Age"],
        y=age_summary["TVPI_median"],
        mode="lines+markers",
        name="Median TVPI",
        line=dict(color=color_b, width=2, dash="dash"),
        yaxis="y2",
    )
)

# Layout and style
fig.update_layout(
    title="Average/Median IRR and TVPI by Fund Age",
    xaxis=dict(title="Fund Age (Years)"),
    yaxis=dict(
        title="IRR (%)",
        color=color_a,
        tickfont=dict(color=color_a),
        side="left",
    ),
    yaxis2=dict(
        title="TVPI",
        color=color_b,
        tickfont=dict(color=color_b),
        overlaying="y",
        side="right",
    ),
    legend=dict(
        x=0,
        y=1.12,  # raised the legend a bit higher
        orientation="h",
        bgcolor="rgba(255,255,255,0)",  # Transparent background
        bordercolor="rgba(0,0,0,0)",  # No border
    ),
    margin=dict(l=60, r=60, t=100, b=60),
    hovermode="x unified",
    template="plotly_white",
)

fig.show()

As expected, we see the lowest `IRR` and `TVPI` for funds that are only a year old. The `IRR` typically accelerates in the second year as initial investments begin to generate cash flows. After this early growth phase, the `IRR` appears to decline slightly but remains at relatively high levels until around year 6.

Meanwhile, `TVPI` tends to increase more gradually since it measures cumulative value creation, including unrealized gains that may take longer to materialize. This slower buildup of `TVPI` highlights the long-term nature of value realization in private equity.

Between years 5 and 10, both `IRR` and `TVPI` generally stabilize as investments mature and more exits occur, delivering realized returns to investors. This phase reflects the core value creation window for most funds.

Notably, the median values reveal the more typical fund experience, smoothing out the effect of a few exceptionally high-performing funds that push the averages upward. This comparison between mean and median offers a clearer picture of broader fund performance trends across different ages.

---

## Point In Time Visualization

Next, we analyze how the benchmark funds, `Apple`, `Banana`, and `Cherry`, compare to their peers at specific points in time. Specifically, we explore the relationship between `IRR` and `TVPI` across private equity funds from three key vintage years: 2012, 2016, and 2019. These vintages are important as they reflect distinct fundraising cycles with varying market conditions and investment opportunities. To provide meaningful context, each benchmark fund is compared against its peer group from the same vintage year.

Using a scatterplot, I plot each fund’s `IRR` (x-axis) against its `TVPI` (y-axis) within their respective vintages. Logarithmic transformations are applied to manage outliers, as detailed earlier. Points are color-coded by vintage for clarity, while the benchmark funds (`Apple`, `Banana`, `Cherry`) are distinctly highlighted with star symbols and labeled to showcase their positions relative to peers.

This visualization facilitates a clear, side-by-side comparison of individual fund performance within vintage cohorts, as well as across different vintages, all within a single view.

In [13]:
# Calculate window limits for log scale
valid_df = combined_df[(combined_df["IRR"] > 0) & (combined_df["TVPI"] > 0)]
log_irr_range = np.log10([valid_df["IRR"].min(), valid_df["IRR"].max()])
log_tvpi_range = np.log10([valid_df["TVPI"].min(), valid_df["TVPI"].max()])

# Map styling attributes for highlighted and non-highlighted points
combined_df["Size"] = combined_df["Highlight"].map({False: 3, True: 25})
combined_df["Symbol"] = combined_df["Highlight"].map({False: "circle", True: "star"})

# Assign colors per vintage
vintages = sorted(combined_df["Vintage"].unique())
color_map = px.colors.qualitative.Plotly
vintage_colors = {v: color_map[i % len(color_map)] for i, v in enumerate(vintages)}

fig = go.Figure()

# For each vintage year
for vintage in vintages:
    color = vintage_colors[vintage]
    vintage_df = combined_df[combined_df["Vintage"] == vintage]

    # For normal points
    normal_df = vintage_df[vintage_df["Highlight"] == False] # Exclude for default view framing
    # Highlighted points keep all regardless of TVPI or IRR
    highlight_df = vintage_df[vintage_df["Highlight"] == True]

    # Normal points (smaller, lower opacity, circle symbol)
    fig.add_trace(
        go.Scatter(
            x=normal_df["IRR"],
            y=normal_df["TVPI"],
            mode="markers",
            marker=dict(
                size=normal_df["Size"],
                symbol=normal_df["Symbol"],
                color=color,
                opacity=0.1,
                line=dict(width=0),
            ),
            name=f"Vintage {vintage}",
            legendgroup=f"Vintage {vintage}",
            showlegend=False,
            hovertemplate=(
                "Fund: %{customdata[0]}<br>" +
                "Vintage: %{customdata[1]}<br>" +
                "IRR: %{x:.2f}<br>" +
                "TVPI: %{y:.2f}<extra></extra>"
            ),
            customdata=np.stack([normal_df["Fund Name"], normal_df["Vintage"]], axis=-1),
        )
    )

    # Dummy trace for legend
    fig.add_trace(
        go.Scatter(
            x=[None], y=[None],
            mode="markers",
            marker=dict(size=10, symbol="circle", color=color, opacity=1.0),
            name=f"Vintage {vintage}",
            legendgroup=f"Vintage {vintage}",
            showlegend=True,
            hoverinfo="skip",
        )
    )

    # Highlighted points (larger, full opacity, star symbol)
    if not highlight_df.empty:
        fig.add_trace(
            go.Scatter(
                x=highlight_df["IRR"],
                y=highlight_df["TVPI"],
                mode="markers+text",
                marker=dict(
                    size=highlight_df["Size"],
                    symbol=highlight_df["Symbol"],
                    color=color,
                    opacity=1.0,
                    line=dict(color="white", width=2),
                ),
                text=highlight_df["Fund Name"],
                textposition="top center",
                name=f"Vintage {vintage} (Highlighted)",
                legendgroup=f"Vintage {vintage}",
                showlegend=False,
                hovertemplate=(
                    "Fund: %{customdata[0]}<br>" +
                    "Vintage: %{customdata[1]}<br>" +
                    "IRR: %{x:.2f}<br>" +
                    "TVPI: %{y:.2f}<extra></extra>"
                ),
                customdata=np.stack([highlight_df["Fund Name"], highlight_df["Vintage"]], axis=-1),
            )
        )

# Update formatting
fig.update_layout(
    title="Comparative Analysis of Benchmark Funds vs Peers (IRR & TVPI)",
    template="plotly_white",
    xaxis=dict(type="log", title="IRR (%)", range=log_irr_range.tolist()),
    yaxis=dict(type="log", title="TVPI", range=log_tvpi_range.tolist()),
)

fig.show()

This scatterplot offers a concise overview of private equity fund performance by vintage year:

- The 2012 vintage exhibits the greatest variability, especially in `TVPI`, where fund age plays a significant role. This is characteristic of a mature vintage, where funds have had ample time to realize value. The wide dispersion reflects a range of investment strategies and outcomes over an extended period.

- The 2016 vintage clusters more tightly, indicating mid-stage value creation as many funds advance through their critical growth phase.

- The 2019 vintage is comparatively younger and less developed, shown by generally lower `TVPI` values, which aligns with the longer horizon required for value generation in private equity.

- Across both `IRR` and `TVPI`, benchmark funds (`Apple`, `Banana`, `Cherry`) consistently land near the middle of their vintage peers.

Overall, this analysis underscores the importance of considering vintage year context when interpreting `IRR` and `TVPI`, and highlights the role of benchmark funds in setting performance expectations within each cohort.


---

## Summary

In this notebook, I've built a flexible ETL pipeline to consolidate and clean Excel exports into a unified dataset of nearly 20,000 private equity fund records. I performed exploratory data analysis and created two visualizations: a time-based view of typical fund performance by age, and a point-in-time `IRR` vs. `TVPI` comparison highlighting benchmark funds (`Apple`, `Banana`, `Cherry`) against their comparable vintages. The approach is modular and scalable for future data additions, designed to support deeper portfolio analytics.

To enable simple replication of this analysis, I've also exported the project dependencies into a `requirements.txt` file below.


In [14]:
!pip freeze > requirements.txt

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=164ec518-eecd-45e0-bbe9-28739515af88' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>